### VSASV

In [ ]:
import torch
import torchaudio
import pandas as pd
import os

class Parquet2Audio_VSASV:
    def __init__(self, save_dir="audio_data"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)

    def process_parquet_folder(self, parquet_folder_path):
        for root, dirs, files in os.walk(parquet_folder_path):
            for file in files:
                if file.endswith(".parquet"):
                    full_path = os.path.join(root, file)
                    print(f"📂 Đang xử lý: {full_path}")
                    self.process_single_parquet(full_path)

    def process_single_parquet(self, parquet_path):
        try:
            df = pd.read_parquet(parquet_path)

            for index, row in df.iterrows():
                try:
                    # Chỉ lấy dữ liệu thật (bonafide), bỏ qua dữ liệu giả (spoof)
                    if 'utt_type' in row and row['utt_type'] != 'bonafide':
                        continue

                    # Speaker ID
                    speaker_id = str(row['label'])

                    # File name
                    filename = os.path.basename(str(row['file']))
                    if not filename.endswith('.wav'):
                        filename = filename.replace('.', '_') + '.wav'

                    # --- XỬ LÝ AUDIO ARRAY ---
                    audio_dict = row['audio']
                    input_array = audio_dict['array']
                    waveform = torch.tensor(input_array, dtype=torch.float32)

                    # Thêm chiều channels (Time,) -> (1, Time)
                    if waveform.dim() == 1:
                        waveform = waveform.unsqueeze(0)

                    # Kiểm tra audio rỗng
                    if waveform.numel() == 0:
                        print(f"⚠️ Audio rỗng tại dòng {index}, bỏ qua.")
                        continue

                    # Lấy sample rate gốc từ dict
                    orig_sr = audio_dict.get('sampling_rate')
                    if orig_sr is None:
                        print(f"⚠️ Cảnh báo: Không tìm thấy sampling_rate ở dòng {index}, bỏ qua.")
                        continue

                    waveform = waveform.clamp(-1.0, 1.0)

                    # Save
                    spk_dir = os.path.join(self.save_dir, speaker_id)
                    os.makedirs(spk_dir, exist_ok=True)
                    save_path = os.path.join(spk_dir, filename)

                    torchaudio.save(save_path, waveform, orig_sr)

                except Exception as e:
                    print(f"❌ Lỗi dòng {index}: {str(e)}")
                    continue

            print(f"✅ Đã xử lý xong file: {parquet_path}")

        except Exception as e:
            print(f"❌ Lỗi đọc file parquet: {str(e)}")


if __name__ == "__main__":
    parquet_folder = r""
    cleaner = Parquet2Audio_VSASV(save_dir="VSASV_wav")
    cleaner.process_parquet_folder(parquet_folder)

### Preprocessing for VoxVietnam

In [ ]:
class Parquet2Audio_Vox:
    def __init__(self, save_dir="audio_data"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)

    def process_parquet_folder(self, parquet_folder_path):
        for root, dirs, files in os.walk(parquet_folder_path):
            for file in files:
                if file.endswith(".parquet"):
                    full_path = os.path.join(root, file)
                    print(f"📂 Đang xử lý file: {file}")
                    self.process_single_parquet(full_path)

    def process_single_parquet(self, parquet_path):
        try:
            df = pd.read_parquet(parquet_path)

            # Lấy tên file parquet để làm tiền tố đặt tên
            parquet_name = os.path.splitext(os.path.basename(parquet_path))[0]

            for index, row in df.iterrows():
                try:
                    # Speaker ID
                    speaker_id = str(row['speaker'])

                    # Tạo tên file: speakerID_parquetName_index.wav
                    filename = f"{speaker_id}_{parquet_name}_{index}.wav"

                    # --- XỬ LÝ AUDIO ARRAY ---
                    audio_dict = row['audio']
                    input_array = audio_dict['array']
                    waveform = torch.tensor(input_array, dtype=torch.float32)

                    if waveform.dim() == 1:
                        waveform = waveform.unsqueeze(0)

                    if waveform.numel() == 0:
                        print(f"⚠️ Audio rỗng tại dòng {index}, bỏ qua.")
                        continue

                    orig_sr = audio_dict.get('sampling_rate')
                    if orig_sr is None:
                        print(f"⚠️ Cảnh báo: Không tìm thấy sampling_rate ở dòng {index}, bỏ qua.")
                        continue

                    waveform = waveform.clamp(-1.0, 1.0)

                    # Save
                    spk_dir = os.path.join(self.save_dir, speaker_id)
                    os.makedirs(spk_dir, exist_ok=True)
                    save_path = os.path.join(spk_dir, filename)

                    torchaudio.save(save_path, waveform, orig_sr)

                except Exception as e:
                    print(f"❌ Lỗi dòng {index}: {str(e)}")
                    continue

            print(f"✅ Xong file parquet: {parquet_path}")

        except Exception as e:
            print(f"❌ Lỗi đọc file parquet {parquet_path}: {str(e)}")


if __name__ == "__main__":
    parquet_folder_path = r""
    cleaner = Parquet2Audio_Vox(save_dir="Vox_train_wav")
    cleaner.process_parquet_folder(parquet_folder_path)